In [15]:
import pandas as pd
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
pd

<module 'pandas' from '/Users/harry/anaconda3/lib/python3.11/site-packages/pandas/__init__.py'>

In [17]:
import os
item_file = os.listdir("item")
full = pd.DataFrame()
for i in item_file:
    temp= pd.read_csv(f"item/{i}")
    # temp = temp[['title','categories','rating','link']]
    full = pd.concat([full,temp])
full

Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  position  \
0              0             0           0         1   
1              1             1           1         2   
2              2             2           2         3   
3              3             3           3         4   
4              4             4           4         5   
..           ...           ...         ...       ...   
65            65            65          65        66   
66            66            66          66        67   
67            67            67          67        68   
68            68            68          68        69   
69            69            69          69        70   

                                                title        asin  \
0   ThinOptics Milano Aluminum Case + Frontpage Ma...  B07FKYPXRR   
1   NOVIVON 6 Pack Reading Glasses Blue Light Bloc...  B0BHSZWS9P   
2   Frameless Reading Glasses Blue Light Blocking ...  B0995TMVZB   
3   Foaincore 12 Count Hobnail Drinking Glasses Se...  B0CN93M199   
4   Gaoye Blue Light Blocking Glasses - 3 Pack Fas...  B085XGKBDT   
..                                                ...         ...   
65  Burt's Bees Lip Balm Easter Basket Stuffers - ...  B00TQ2X690   
66  Burt's Bees Lip Balm Easter Basket Stuffers - ...  B01N2UBGQJ   
67  Physicians Formula Murumuru Butter Blush Vinta...  B06XJR7271   
68  Burt's Bees Rescue Lip Relief Lip Balm, With S...  B0B8F38RGD   
69  8-Pack Lip Balm Gift Set by Naturistick. Assor...  B073Z7525P   

                                                 link categories  \
0   https://www.amazon.com/sspa/click?ie=UTF8&spc=...    Outwear   
1   https://www.amazon.com/sspa/click?ie=UTF8&spc=...    Outwear   
2   https://www.amazon.com/sspa/click?ie=UTF8&spc=...    Outwear   
3   https://www.amazon.com/sspa/click?ie=UTF8&spc=...    Outwear   
4   https://www.amazon.com/Gaoye-Blocking-Computer...    Outwear   
..                                                ...        ...   
65  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  Cosmetics   
66  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  Cosmetics   
67  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  Cosmetics   
68  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  Cosmetics   
69  https://www.amazon.com/sspa/click?ie=UTF8&spc=...  Cosmetics   

                                                image is_prime  ...  \
0   https://m.media-amazon.com/images/I/51G9zMrdw9...     True  ...   
1   https://m.media-amazon.com/images/I/71PNWMcjXy...     True  ...   
2   https://m.media-amazon.com/images/I/71an4coU5C...     True  ...   
3   https://m.media-amazon.com/images/I/91fLQfs11c...     True  ...   
4   https://m.media-amazon.com/images/I/61Qpkx-Obh...     True  ...   
..                                                ...      ...  ...   
65  https://m.media-amazon.com/images/I/81fuHPSief...     True  ...   
66  https://m.media-amazon.com/images/I/813FRSgVu+...     True  ...   
67  https://m.media-amazon.com/images/I/81oNXOaYLp...     True  ...   
68  https://m.media-amazon.com/images/I/71tFv7gg0G...     True  ...   
69  https://m.media-amazon.com/images/I/81M5c04ECp...     True  ...   

          unit_price  is_carousel  \
0                NaN          NaN   
1                NaN          NaN   
2                NaN          NaN   
3   $5.00$5.00/Count          NaN   
4                NaN          NaN   
..               ...          ...   
65               NaN         True   
66               NaN         True   
67               NaN         True   
68               NaN         True   
69  $1.25$1.25/Count         True   

                                             carousel  \
0                                                 NaN   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
..                                           

In [18]:
full['bestseller']

0                                                   NaN
1                                                   NaN
2                                                   NaN
3                                                   NaN
4                                                   NaN
                            ...                        
65    {'link': 'https://www.amazon.com/gp/bestseller...
66                                                  NaN
67                                                  NaN
68                                                  NaN
69                                                  NaN
Name: bestseller, Length: 559, dtype: object

In [19]:
# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Assuming data_t is already defined as per your description
data = full.copy()

# Function to extract subcategory using NER
def extract_subcategory_with_ner_and_nouns(title):
    doc = nlp(title)
    # Initialize an empty list to hold entities and nouns
    subcategories = []
    
    # Extract entities specifically if they are PRODUCT or ORG
    for ent in doc.ents:
        if ent.label_ in ["PRODUCT", "ORG"]:
            subcategories.append(ent.text)
    
    # Extract nouns not already captured as part of entities
    nouns = [token.text for token in doc if token.pos_ == "NOUN" and token.text not in subcategories]
    subcategories.extend(nouns)
    
    # Join the list of entities and nouns into a single string
    return " ".join(subcategories)
# Apply NER function to extract subcategories
data['subcategory'] = data['title'].apply(extract_subcategory_with_ner_and_nouns)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(data['title'], data['categories'], test_size=0.25, random_state=42)

# Pipeline for classification
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Predict and evaluate
predicted_labels = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, predicted_labels)}")


Accuracy: 0.9785714285714285


In [36]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
data = full.copy()
# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Function to extract subcategory using NER and nouns
def extract_subcategory_with_ner_and_nouns(title):
    doc = nlp(title)
    subcategories = []
    for ent in doc.ents:
        if ent.label_ in ["PRODUCT", "ORG"]:
            subcategories.append(ent.text)
    nouns = [token.text for token in doc if token.pos_ == "NOUN" and token.text not in subcategories]
    subcategories.extend(nouns)
    return " ".join(subcategories)

# Function to recommend top 10 similar items based on a new title, considering color
def recommend_similar_items(new_title, data):
    # Transform the dataset titles and the new title into TF-IDF vectors
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(data['title'])
    new_title_vector = tfidf_vectorizer.transform([new_title])
    
    # Calculate the cosine similarity
    cosine_similarities = cosine_similarity(new_title_vector, tfidf_matrix).flatten()
    data['similarity'] = cosine_similarities
    
    # Predict the category of the new title
    predicted_category = model.predict([new_title])[0]
    filtered_items = data[data['categories'] == predicted_category]

    # Extract color and refine recommendation based on color if specified
    doc = nlp(new_title)
    colors = [token.text for token in doc if token.pos_ == "ADJ"]  # Assuming adjectives as potential colors
    if colors:
        filtered_items = filtered_items[filtered_items['colour_tag'].str.contains('|'.join(colors), case=False, na=False)]

    # Sort items by similarity score and review score, and select top 10
    top_10_similar_items = filtered_items.sort_values(by=['similarity', 'rating'], ascending=False).head(10)
    
    return top_10_similar_items[['title', 'link']]

# Example of pipeline and data preparation would be included here.

# Example usage:
new_title = "I want to find the blue iphone"
recommended_items = recommend_similar_items(new_title, data)


In [37]:
recommended_items

title  \
17  Apple iPhone 14, 256GB, Blue - Unlocked (Renewed)   
67  Apple iPhone 15 Plus, 256GB, Blue - Unlocked (...   
6   Apple iPhone 12, 64GB, Blue - Fully Unlocked (...   
90  Apple iPhone XR, 128GB, Blue - For T-Mobile (R...   
16  Apple iPhone 14 Plus, 128GB, Blue - Unlocked (...   
24  Apple iPhone 14 Plus, 128GB, Blue - Unlocked (...   
75  Apple iPhone 12 Mini, 64GB, Blue - Unlocked (R...   
89  Apple iPhone 12 Mini, 128GB, Blue - Unlocked (...   
10  Apple iPhone XR, US Version, 64GB, Blue - Unlo...   
11  Apple iPhone 13 Pro, 128GB, Sierra Blue - Unlo...   

                                                 link  
17  https://www.amazon.com/Apple-iPhone-14-256GB-B...  
67  https://www.amazon.com/Apple-iPhone-15-Plus-25...  
6   https://www.amazon.com/Apple-iPhone-12-64GB-Bl...  
90  https://www.amazon.com/Apple-iPhone-XR-128GB-B...  
16  https://www.amazon.com/Apple-iPhone-Plus-128GB...  
24  https://www.amazon.com/Apple-iPhone-Plus-128GB...  
75  https://www.amazon.com/Apple-iPhone-12-Mini-64...  
89  https://www.amazon.com/Apple-iPhone-12-Mini-12...  
10  https://www.amazon.com/Apple-iPhone-XR-Fully-U...  
11  https://www.amazon.com/Apple-iPhone-13-Pro-Sie...